# Multiple dispatch in Julia

# The expression problem

> To which degree can your application be structured in such a way that both the data model and the set of virtual operations over it can be extended without the need to modify existing code, without the need for code repetition and without runtime type errors.


(Torgersen M. The expression problem revisited.)

**types:**       `int`, `float`, `double`, `vector`, ...  

**operations:**  `+`, `-`, `*`, `/`, `%`, `push`, `pop`, ...

# Example


### Polynomial class (Alice nice library)
* evaluation
* adding/substraction
* multiplication
* differentiation
* etc.


### "Polynomial geometry" (Bob's awesome extension)
* inner products
* affine transformations
* "angles" between polynomials
* etc.

# Solutions from OOP


## Extending the exsiting library
* convincing Alice to include Bob's functionality
* who does the maintenance? Alice? Bob? 


## Inheritence
* Bob inherit from Alice polynomial class
* the new class needs a new name
* what if Alice wants to change her lib? (reverse API problem)
* what if Charly wants to extent this as well? Multiple inheritence? 

## How to introduce new types the existing operations can be applyed on?
* easy in object oriented languages
* hard in functional languages



## How to introduce new operations, which can be applyed on the existing types?
* hard in object oriented languages
* easy in functional languages

## How to do both?
* the actual expression problem

# Solution in Julia

# ➡️ Multiple dispatch

# Example

In [ ]:
f(x::Int64) = "This is the integer version: $x"
f(x::Float64) = "This is the float version: $x"
f(x) = "This is the generic fallback: $x";

In [ ]:
@show f(1);
@show f(1.0);
@show f("🐈");

# Toolbox addition

In [ ]:
@code_warntype f(1)

In [ ]:
@code_warntype f(1.0)

➡️ different outputs for different *input types*

# This is called (single) dispatch!
* the actual method is selected w.r.t. the input type



## What if we have more arguments?

In [ ]:
@show 1*1
@show 1.0*1.0
@show "🐱"*"🐱";

In [ ]:
@code_warntype 1*1

In [ ]:
@code_warntype 1.0*1.0

In [ ]:
@code_warntype 1*1.0

In [ ]:
@code_warntype "🐱"*"🐱"

# This is multiple dispatch!
* the actual method is selected considering the most specialised type for **each** argument

# A solution to the expression problem in Julia

## Example: inner sum
credits: Stefan Karpinski

In [ ]:
using LinearAlgebra
using BenchmarkTools

function inner_sum(A,vs)
    t = zero(eltype(A))
    for v in vs
        t += inner(v,A,v) # dispatch here
    end
    return t
end

inner(v,A,w) = dot(v,A*w) # much generic!

➡️ Stefan's pro tip: to write highly generic code, just leave off all the types

In [ ]:
A = rand(10,10)
vs = [rand(10) for _ in 1:4]

@show inner_sum(A,vs)
@benchmark inner_sum($A,$vs)

* very generic!
* adds new functions to existing types!

# The one-hot vector

$v = (0,\dots,0,1,0,\dots,0)$

In [ ]:
struct OneHotVector <: AbstractVector{Bool}
    hot::Int64
    len::Int64
end

In [ ]:
Base.size(v::OneHotVector) = (v.len,)
Base.getindex(v::OneHotVector,idx::Integer) = (idx==v.hot)

In [ ]:
A = rand(10,10)
one_hots = [OneHotVector(rand(1:10),10) for _ in 1:4]

@show inner_sum(A,one_hots)
@benchmark inner_sum($A,$one_hots)

* uses only generic fallbacks!
* not that impressive!

# How efficient can it be?

# Matrix multiplication

## Standard matmul

$A*v := \left(\sum_{n=1}^N A_{mn}v_n\right)_{m=1}^N$




## The one-hot matmul

$A*v := \left(A_{m n_{\mathrm{hot}}}\right)_{m=1}^N$

In [ ]:
import Base: *

*(A::AbstractMatrix,v::OneHotVector) = A[:, v.hot];

In [ ]:
A = rand(10,10)
one_hots = [OneHotVector(rand(1:10),10) for _ in 1:4]

@show inner_sum(A,one_hots)
@benchmark inner_sum($A,$one_hots)

# Inner product for one-hots
## Standard inner

$v*A*w = \sum_{n=1}^N \sum_{m=1}^N v_n A_{mn} w_m$




## The one-hot inner

$v*A*w = A_{n_{\mathrm{hot}}m_{\mathrm{hot}}}$

In [ ]:
function inner(v::OneHotVector,A::AbstractMatrix, w::OneHotVector)
    return A[v.hot,w.hot]
end

In [ ]:
A = rand(10,10)
one_hots = [OneHotVector(rand(1:10),10) for _ in 1:4]

@show inner_sum(A,one_hots)
@benchmark inner_sum($A,$one_hots)

# Summary


## expression problem
* hard to solve with build-ins of most functional and object oriented languages
* possible solution: multiple dispatch

## Julia's multiple dispatch
* adding new functions to existing types is plain simple (e.g. `inner` and `inner_sum`)
* adding new types to existing functionality is also plain simple (e.g. `OneHotVector` on `Base.size`, `Base.getindex`)
* can be arbitrary efficient (e.g. specialising `*`, or `inner` on `OneHotVector`)